In [ ]:
#@title DeFooocus
#@markdown **Launch the interface DeFocus (Fooocus fork)** | You need to connect with T4/A100/V100
#@markdown ****
#@markdown *Attention!* When working in the interface with the FaceSwap and CPDS controlnet, crashes are possible; it is also recommended to work in *Extreme speed* mode for additional stability. When working with the ImagePrompt and PyraCanny controls, 85% of the work will be stable.
#@markdown ****

print("[DeFooocus] Preparing ...")

theme = "dark" #@param ["dark", "light"]
preset = "deafult" #@param ["deafult", "realistic", "anime", "lcm", "sai", "turbo", "lighting", "hypersd", "playground_v2.5", "dpo", "spo", "sd1.5"]
advenced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16" #@param {type: "string"}

if preset != "deafult":
  args = f"{advenced_args} --theme {theme} --preset {preset}"
else:
  args = f"{advenced_args} --theme {theme}"

!pip install -q pygit2==1.12.2
%cd /content
!git clone https://github.com/ehristoforu/DeFooocus.git
%cd /content/DeFooocus
!pip install -q -r requirements_versions.txt


In [ ]:
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
import gdown
from gdown.download import get_url_from_gdrive_confirmation
import six
import re
from tqdm import tqdm

#@markdown # Model Download/Load

#@markdown Models are downloaded or moved to the `Fooocus/models/checkpoints/` folder. If they already exist in that folder you don't need to reload them.

#@markdown ---
PATH_to_MODEL = "" #@param {type:"string"}
#@markdown - Insert the full Google Drive path of your custom model

#@markdown OR
MODEL_LINK = "" #@param {type:"string"}
#@markdown - Insert the URL of the model you'd like to download


def getsrc(url):
    parsed_url = urlparse(url)
    if parsed_url.netloc == 'civitai.com':
        src='civitai'
    elif parsed_url.netloc == 'drive.google.com':
        src='gdrive'
    elif parsed_url.netloc == 'huggingface.co':
        src='huggingface'
    else:
        src='others'
    return src

src=getsrc(MODEL_LINK)

def get_name(url, gdrive):
    if not gdrive:
        response = requests.get(url, allow_redirects=False)
        if "Location" in response.headers:
            redirected_url = response.headers["Location"]
            quer = parse_qs(urlparse(redirected_url).query)
            if "response-content-disposition" in quer:
                disp_val = quer["response-content-disposition"][0].split(";")
                for vals in disp_val:
                    if vals.strip().startswith("filename="):
                        filenm=unquote(vals.split("=", 1)[1].strip())
                        return filenm.replace("\"","")
    else:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"}
        lnk="https://drive.google.com/uc?id={id}&export=download".format(id=url[url.find("/d/")+3:url.find("/view")])
        res = requests.session().get(lnk, headers=headers, stream=True, verify=True)
        res = requests.session().get(get_url_from_gdrive_confirmation(res.text), headers=headers, stream=True, verify=True)
        content_disposition = six.moves.urllib_parse.unquote(res.headers["Content-Disposition"])
        filenm = re.search(r"filename\*=UTF-8''(.*)", content_disposition).groups()[0].replace(os.path.sep, "_")
        return filenm


def dwn(url, dst, msg):
    file_size = None
    req = Request(url, headers={"User-Agent": "torch.hub"})
    u = urlopen(req)
    meta = u.info()
    if hasattr(meta, 'getheaders'):
        content_length = meta.getheaders("Content-Length")
    else:
        content_length = meta.get_all("Content-Length")
    if content_length is not None and len(content_length) > 0:
        file_size = int(content_length[0])

    with tqdm(total=file_size, disable=False, mininterval=0.5,
              bar_format=msg+' |{bar:20}| {percentage:3.0f}%') as pbar:
        with open(dst, "wb") as f:
            while True:
                buffer = u.read(8192)
                if len(buffer) == 0:
                    break
                f.write(buffer)
                pbar.update(len(buffer))
            f.close()


if (PATH_to_MODEL !=''):
  if os.path.exists(str(PATH_to_MODEL)):
    inf('\u2714 Using the trained model.','success', '200px')

  else:
      while not os.path.exists(str(PATH_to_MODEL)):
        inf('\u2718 Wrong path, use the colab file explorer to copy the path : ','danger', "400px")
        PATH_to_MODEL=input()
      if os.path.exists(str(PATH_to_MODEL)):
        inf('\u2714 Using the custom model.','success', '200px')

  model=PATH_to_MODEL

elif MODEL_LINK != "":

      if src=='civitai':
         modelname=get_name(MODEL_LINK, False)
         model=f'/content/gdrive/{mainpth}/{folderpath}/Fooocus/models/checkpoints/{modelname}'
         if not os.path.exists(model):
            dwn(MODEL_LINK, model, 'Downloading the custom model')
            clear_output()
         else:
            inf('\u2714 Model already exists','primary', '300px')
      elif src=='gdrive':
         modelname=get_name(MODEL_LINK, True)
         model=f'/content/gdrive/{mainpth}/{folderpath}/Fooocus/models/checkpoints/{modelname}'
         if not os.path.exists(model):
            gdown.download(url=MODEL_LINK, output=model, quiet=False, fuzzy=True)
            clear_output()
         else:
            inf('\u2714 Model already exists','primary', '300px')
      else:
         modelname=os.path.basename(MODEL_LINK)
         model=f'/content/gdrive/{mainpth}/{folderpath}/Fooocus/models/checkpoints/{modelname}'
         if not os.path.exists(model):
            gdown.download(url=MODEL_LINK, output=model, quiet=False, fuzzy=True)
            clear_output()
         else:
            inf('\u2714 Model already exists','primary', '700px')

      if os.path.exists(model) and os.path.getsize(model) > 1810671599:
        inf('\u2714 Model downloaded, using the custom model.','success', '300px')
      else:
        !rm model
        inf('\u2718 Wrong link, check that the link is valid','danger', "300px")

#@markdown ---

In [ ]:
#@title Run
print("[DeFooocus] Starting ...")
!python entry_with_update.py $args